In [4]:
from huggingface_hub import login

# hugging face api 토큰 입력
api_token = "****"
login(api_token)

In [5]:
import torch
import wandb

from sklearn.model_selection import train_test_split

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline, 
    Trainer
)
from transformers.integrations import WandbCallback
from trl import DataCollatorForCompletionOnlyLM
import evaluate

# 모델과 토크나이저 불러오기 
model_name = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_name,
    use_cache=False,    # 캐시 사용 안함
    device_map="auto",  # 장치 설정, GPU가 있으면 GPU가 선택되고, GPU가 없으면 CPU가 선택
    torch_dtype=torch.bfloat16,  # 파이토치에서 사용할 데이터 형식, bfloat16: 16비트는 사용하는 부동소수점 형식, float16보다 더 넓은 범위 수 표현, 수치 안정성
    low_cpu_mem_usage=True,      # CPU 메모리를 최소화 하면서 모델을 불러옴
    attn_implementation="eager",  # 어떤 어텐션 메커니즘을 사용할건지, eager: 기본 메커니즘, 저사양용
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [6]:
# 데이터셋 준비
import datasets
dataset = datasets.load_dataset("jaehy12/news3")  # original에는 뉴스의 원문이, summary에는 해당 원문을 한두 문장 정도로 요약한 내용이 들어있음
element = dataset["train"][1]
element

dataset (1).csv:   0%|          | 0.00/77.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24300 [00:00<?, ? examples/s]

{'original': ' “동선 봐도 도움이 안 되는 것 같아요.\n  어딘지 알아야 피해가고 조심할 텐데요.\n ”“확진자가 다녀간 곳은 방역 마쳤는데 동선 공개는 그 부근 일대를 싹 다 죽이자는 것 같아요.\n ” 최근 부천시 페이스북 신종 코로나바이러스 감염증(코로나19) 관련 게시물에는 확진자 동선 공개 범위를 성토하는 댓글이 여럿 달렸다.\n  물류센터·교회 발 코로나19 확산이 지역사회 N차 감염으로 이어지면서 확진자 동선을 확대 공개해야 한다는 주장이 제기된 가운데 사생활 침해 등을 근거로 반대하는 의견도 나오면서 동선공개 논란에 다시 불이 붙는 모양새다.\n  지난 2월 코로나19 초기엔 확진자 동선이 대부분 공개됐다.\n  쇼핑몰을 방문한 확진자가 시간대별로 어느 매장을 찾았는지 등의 동선이 지자체 소셜네트워크서비스(SNS)에 게재됐다.\n  이후 사생활 침해 논란이 일자 질본은 감염병 예방에 필요한 정보에 한해 확진자 정보를 공개하라는 권고사항을 발표했다.\n  권고사항에는 증상 발생 2일 전부터 격리일까지의 시간, 감염을 우려할 만큼의 확진자와의 접촉이 일어난 장소 및 수단 등을 공개한다는 내용이 담겼다.\n  해당 공간 내 모든 접촉자가 파악되면 공개하지 않을 수 있다는 단서도 추가됐다.\n  확진자가 마지막 접촉자와 만날 일로부터 14일이 지나면 공개한 동선을 삭제할 것도 권고했다.\n  확진 늘자 동선공개 확대 나선 지자체 물류센터·교회 등 집단감염에 이어 감염경로가 특정되지 않은 확진 사례가 늘면서 일부 지자체는 동선 공개 범위를 확대하고 있다.\n  김포시는 지난 2일부터 확진자 이동 경로에 따른 동선 공개 원칙을 제한적으로 확대했다.\n  시민 우려 불식을 위해 확진자 방문으로 접촉자가 다수 발생하고 확산이 우려되는 장소는 상호를 공개하기로 한 것이다.\n  앞서 부천 118번 확진자인 A씨(31)가 한 제약회사 영업사원인 사실이 알려지면서 SNS에 그가 다닌 병원 정보가 공유됐다.\n  장덕천 부천시장은 “SNS에 A씨가 평소

In [7]:
# Gemma 모델 기본 기능 확인하기
# 1. 키워드 추출 기능 확인
input_text = """
다음 텍스트를 한국어로 간단히 요약해주세요: \n 김해국제공항에서 발생한 에어부산(BX391편) 여객기 화재 사고 원인을 규명하기 위한 조사가 한창인 가운데 에어부산이
기내 화재 위험 최소화 정책을 발표했다. 해당 정책에는 휴대 수하물 내 보조배터리 소지 유무를 사전 확인·통제하고 안내방송 확대와 객실승무원 훈련을 강화하는 내용 등이 담겼다.
에어부산은 4일 보조배터리 기내 탑재 사례가 증가하며 발열 등 이상 현상 발생 빈도가 높아진 데 따라 이 같은 정책을 수립했다고 밝혔다. 에어부산이 내놓은 정책에서 주목할 부분은
휴대 수하물 내 보조배터리 소지 유무를 사전 확인하는 절차다. 에어부산은 탑승구에서 휴대 수하물 내 배터리 소지 유무를 사전 확인하고, 기내에서는 탑승구에서 확인을 완료한 수하물만
선반에 보관될 수 있도록 통제를 강화할 방침이다. 이는 탑승구에서 휴대 수하물 내 배터리가 있는지를 점검해 스티커나 택(TAG) 등의 별도 표식을 부착하고 기내에서는 표식이 부착된
수하물만 선반 보관이 가능하도록 허용하겠다는 의미다. 해당 정책은 오는 7일부터 일부 노선에서 시범 운영을 거쳐 전 노선으로 확대 운영할 계획이다. 이 외에도 예약·발권과 탑승수속
단계에서 기존 안내에 더해 보조배터리 기내 선반 탑재 금지에 대한 동의 절차를 시행하고, 출발 1일 전 예약 고객 대상 별도의 안내 문자를 발송하는 등 안내도 강화한다.
기내 안내 방송도 기존 2회에서 3회로 확대 시행하기로 했다. 에어부산은 한층 구체적이고 명확한 내용으로 기내 안내 방송문을 보완해 지난달 말부터 변경 적용한 바 있다.
이 외에도 객실승무원 훈련도 강화할 방침이다. 또 배터리 화재 발생 시 열폭주와 폭발 위험을 최소화할 수 있는 장비(Lithium battery fire containment pouch)와 방화장갑
등을 구매·구비할 계획도 밝혔다. 최근 조사당국은 화재 원인을 규명하기 위해 합동 감식에 나서는 등 사고 관련 조사 진행이 한창이다. 항공·철도사고조사위원회와 프랑스
사고조사당국(BEA), 국립과학수사연구원, 경찰 과학수사대, 소방으로 구성된 합동조사팀은 현장에서 수집한 증거물을 국립과학수사연구원 등으로 옮겨 정밀 감식을 진행 중인 것으로
알려졌다. 이 외에도 사조위는 관계자와 목격자 진술을 받고 폐쇄회로(CC)TV 영상과 블랙박스를 분석하는 등의 조사를 진행 중이다. 정병섭 에어부산 대표이사는
"기내 화재에 대한 손님들의 우려와 불안을 해소하고 안전한 여행 환경을 확보하기 위해 에어부산이 선제적으로 강화 정책을 수립하게 됐다"며 "사내 종사자들의 안전의식 고취와
역량 강화를 비롯해 항공 안전 문화 확산을 위해서도 지속적으로 노력해 나가겠다"고 밝혔다. 앞서 지난달 28일 오후 부산에서 홍콩으로 운항하려던 에어부산 BX391편은 김해공항
계류장에 머무르던 중 항공기 내부 뒤편에서 불이 나기 시작해 동체 상부를 태웠다. 당시 탑승했던 170명 승객과 6명의 승무원은 비상 슬라이드를 통해 탈출했으며,
이 과정에서 7명이 경상을 입었다.
"""

In [8]:
def change_inference_chat_format(input_text):  # 모델이 학습할 때 사용한 대화 형식을 적용하기 위해 위의 로우 데이터를 대화 형식 포맷으로 변경하는 함수
    return[
        {"role": "user", "content": f"{input_text}"},
        {"role": "assistant", "content": """
            김해국제공항에서 발생한 에어부산(BX391편) 여객기 화재 사고 원인을 규명하기 위한 조사가 한창인 가운데 에어부산이
            기내 화재 위험 최소화 정책을 발표했다.""" },
        {"role": "user", "content": "중요한 키워드 5개를 뽑아주세요."},
        {"role": "assistant", "content": ""}
    ]

prompt = change_inference_chat_format(input_text)
prompt

[{'role': 'user',
  'content': '\n다음 텍스트를 한국어로 간단히 요약해주세요: \n 김해국제공항에서 발생한 에어부산(BX391편) 여객기 화재 사고 원인을 규명하기 위한 조사가 한창인 가운데 에어부산이\n기내 화재 위험 최소화 정책을 발표했다. 해당 정책에는 휴대 수하물 내 보조배터리 소지 유무를 사전 확인·통제하고 안내방송 확대와 객실승무원 훈련을 강화하는 내용 등이 담겼다.\n에어부산은 4일 보조배터리 기내 탑재 사례가 증가하며 발열 등 이상 현상 발생 빈도가 높아진 데 따라 이 같은 정책을 수립했다고 밝혔다. 에어부산이 내놓은 정책에서 주목할 부분은\n휴대 수하물 내 보조배터리 소지 유무를 사전 확인하는 절차다. 에어부산은 탑승구에서 휴대 수하물 내 배터리 소지 유무를 사전 확인하고, 기내에서는 탑승구에서 확인을 완료한 수하물만\n선반에 보관될 수 있도록 통제를 강화할 방침이다. 이는 탑승구에서 휴대 수하물 내 배터리가 있는지를 점검해 스티커나 택(TAG) 등의 별도 표식을 부착하고 기내에서는 표식이 부착된\n수하물만 선반 보관이 가능하도록 허용하겠다는 의미다. 해당 정책은 오는 7일부터 일부 노선에서 시범 운영을 거쳐 전 노선으로 확대 운영할 계획이다. 이 외에도 예약·발권과 탑승수속\n단계에서 기존 안내에 더해 보조배터리 기내 선반 탑재 금지에 대한 동의 절차를 시행하고, 출발 1일 전 예약 고객 대상 별도의 안내 문자를 발송하는 등 안내도 강화한다.\n기내 안내 방송도 기존 2회에서 3회로 확대 시행하기로 했다. 에어부산은 한층 구체적이고 명확한 내용으로 기내 안내 방송문을 보완해 지난달 말부터 변경 적용한 바 있다.\n이 외에도 객실승무원 훈련도 강화할 방침이다. 또 배터리 화재 발생 시 열폭주와 폭발 위험을 최소화할 수 있는 장비(Lithium battery fire containment pouch)와 방화장갑\n등을 구매·구비할 계획도 밝혔다. 최근 조사당국은 화재 원인을 규명하기 위해 합동 감식에 나서는 등 사고 관

In [9]:
# 토크나이저 초기화 및 적용
inputs = tokenizer.apply_chat_template(prompt, tokenize=True,
                                       add_generation_prompt=True, return_tensors="pt").to(model.device)
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
다음 텍스트를 한국어로 간단히 요약해주세요: 
 김해국제공항에서 발생한 에어부산(BX391편) 여객기 화재 사고 원인을 규명하기 위한 조사가 한창인 가운데 에어부산이
기내 화재 위험 최소화 정책을 발표했다. 해당 정책에는 휴대 수하물 내 보조배터리 소지 유무를 사전 확인·통제하고 안내방송 확대와 객실승무원 훈련을 강화하는 내용 등이 담겼다.
에어부산은 4일 보조배터리 기내 탑재 사례가 증가하며 발열 등 이상 현상 발생 빈도가 높아진 데 따라 이 같은 정책을 수립했다고 밝혔다. 에어부산이 내놓은 정책에서 주목할 부분은
휴대 수하물 내 보조배터리 소지 유무를 사전 확인하는 절차다. 에어부산은 탑승구에서 휴대 수하물 내 배터리 소지 유무를 사전 확인하고, 기내에서는 탑승구에서 확인을 완료한 수하물만
선반에 보관될 수 있도록 통제를 강화할 방침이다. 이는 탑승구에서 휴대 수하물 내 배터리가 있는지를 점검해 스티커나 택(TAG) 등의 별도 표식을 부착하고 기내에서는 표식이 부착된
수하물만 선반 보관이 가능하도록 허용하겠다는 의미다. 해당 정책은 오는 7일부터 일부 노선에서 시범 운영을 거쳐 전 노선으로 확대 운영할 계획이다. 이 외에도 예약·발권과 탑승수속
단계에서 기존 안내에 더해 보조배터리 기내 선반 탑재 금지에 대한 동의 절차를 시행하고, 출발 1일 전 예약 고객 대상 별도의 안내 문자를 발송하는 등 안내도 강화한다.
기내 안내 방송도 기존 2회에서 3회로 확대 시행하기로 했다. 에어부산은 한층 구체적이고 명확한 내용으로 기내 안내 방송문을 보완해 지난달 말부터 변경 적용한 바 있다.
이 외에도 객실승무원 훈련도 강화할 방침이다. 또 배터리 화재 발생 시 열폭주와 폭발 위험을 최소화할 수 있는 장비(Lithium battery fire containment pouch)와 방화장갑
등을 구매·구비할 계획도 밝혔다. 최근 조사당국은 화재 원인을 규명하기 위해 합동 감식에 나서는 등 사고 관련 조사 진행이 한창이다. 항공·철도사고조사위원회와 프랑스
사고조사당국

In [10]:
# 2. 요약 기능 확인
def change_inference_chat_format(input_text):
    return[
        {"role": "user", "content": f"{input_text}"},
        {"role": "assistant", "content": "한국어 요약:\n"}
    ]

# chat template 적용
prompt = change_inference_chat_format(input_text)

# 생성
inputs = tokenizer.apply_chat_template(prompt, tokenize=True,
                                       add_generation_prompt=True, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=256, use_cache=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
다음 텍스트를 한국어로 간단히 요약해주세요: 
 김해국제공항에서 발생한 에어부산(BX391편) 여객기 화재 사고 원인을 규명하기 위한 조사가 한창인 가운데 에어부산이
기내 화재 위험 최소화 정책을 발표했다. 해당 정책에는 휴대 수하물 내 보조배터리 소지 유무를 사전 확인·통제하고 안내방송 확대와 객실승무원 훈련을 강화하는 내용 등이 담겼다.
에어부산은 4일 보조배터리 기내 탑재 사례가 증가하며 발열 등 이상 현상 발생 빈도가 높아진 데 따라 이 같은 정책을 수립했다고 밝혔다. 에어부산이 내놓은 정책에서 주목할 부분은
휴대 수하물 내 보조배터리 소지 유무를 사전 확인하는 절차다. 에어부산은 탑승구에서 휴대 수하물 내 배터리 소지 유무를 사전 확인하고, 기내에서는 탑승구에서 확인을 완료한 수하물만
선반에 보관될 수 있도록 통제를 강화할 방침이다. 이는 탑승구에서 휴대 수하물 내 배터리가 있는지를 점검해 스티커나 택(TAG) 등의 별도 표식을 부착하고 기내에서는 표식이 부착된
수하물만 선반 보관이 가능하도록 허용하겠다는 의미다. 해당 정책은 오는 7일부터 일부 노선에서 시범 운영을 거쳐 전 노선으로 확대 운영할 계획이다. 이 외에도 예약·발권과 탑승수속
단계에서 기존 안내에 더해 보조배터리 기내 선반 탑재 금지에 대한 동의 절차를 시행하고, 출발 1일 전 예약 고객 대상 별도의 안내 문자를 발송하는 등 안내도 강화한다.
기내 안내 방송도 기존 2회에서 3회로 확대 시행하기로 했다. 에어부산은 한층 구체적이고 명확한 내용으로 기내 안내 방송문을 보완해 지난달 말부터 변경 적용한 바 있다.
이 외에도 객실승무원 훈련도 강화할 방침이다. 또 배터리 화재 발생 시 열폭주와 폭발 위험을 최소화할 수 있는 장비(Lithium battery fire containment pouch)와 방화장갑
등을 구매·구비할 계획도 밝혔다. 최근 조사당국은 화재 원인을 규명하기 위해 합동 감식에 나서는 등 사고 관련 조사 진행이 한창이다. 항공·철도사고조사위원회와 프랑스
사고조사당국

In [11]:
# 3. 키워드 추출 및 요약 기능 확인
def change_inference_chat_format(input_text):
    return[
        {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약하고, 관련된 5개의 키워드를 추출해주세요:\n{input_text}"},
        {"role": "assistant", "content": ""}
    ]

# chat template 적용
prompt = change_inference_chat_format(input_text)

# 생성
inputs = tokenizer.apply_chat_template(prompt, tokenize=True,
                                       add_generation_prompt=True, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=256, use_cache=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
다음 텍스트를 한국어로 간단히 요약하고, 관련된 5개의 키워드를 추출해주세요:

다음 텍스트를 한국어로 간단히 요약해주세요: 
 김해국제공항에서 발생한 에어부산(BX391편) 여객기 화재 사고 원인을 규명하기 위한 조사가 한창인 가운데 에어부산이
기내 화재 위험 최소화 정책을 발표했다. 해당 정책에는 휴대 수하물 내 보조배터리 소지 유무를 사전 확인·통제하고 안내방송 확대와 객실승무원 훈련을 강화하는 내용 등이 담겼다.
에어부산은 4일 보조배터리 기내 탑재 사례가 증가하며 발열 등 이상 현상 발생 빈도가 높아진 데 따라 이 같은 정책을 수립했다고 밝혔다. 에어부산이 내놓은 정책에서 주목할 부분은
휴대 수하물 내 보조배터리 소지 유무를 사전 확인하는 절차다. 에어부산은 탑승구에서 휴대 수하물 내 배터리 소지 유무를 사전 확인하고, 기내에서는 탑승구에서 확인을 완료한 수하물만
선반에 보관될 수 있도록 통제를 강화할 방침이다. 이는 탑승구에서 휴대 수하물 내 배터리가 있는지를 점검해 스티커나 택(TAG) 등의 별도 표식을 부착하고 기내에서는 표식이 부착된
수하물만 선반 보관이 가능하도록 허용하겠다는 의미다. 해당 정책은 오는 7일부터 일부 노선에서 시범 운영을 거쳐 전 노선으로 확대 운영할 계획이다. 이 외에도 예약·발권과 탑승수속
단계에서 기존 안내에 더해 보조배터리 기내 선반 탑재 금지에 대한 동의 절차를 시행하고, 출발 1일 전 예약 고객 대상 별도의 안내 문자를 발송하는 등 안내도 강화한다.
기내 안내 방송도 기존 2회에서 3회로 확대 시행하기로 했다. 에어부산은 한층 구체적이고 명확한 내용으로 기내 안내 방송문을 보완해 지난달 말부터 변경 적용한 바 있다.
이 외에도 객실승무원 훈련도 강화할 방침이다. 또 배터리 화재 발생 시 열폭주와 폭발 위험을 최소화할 수 있는 장비(Lithium battery fire containment pouch)와 방화장갑
등을 구매·구비할 계획도 밝혔다. 최근 조사당국은 화재 원인을 규명하기 위해 합동 감식에 나서는 

In [9]:
# 데이터셋에서 키워드 데이터 생성
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")
def key_word_prompt(input_text, summary_text):
    return [
    {"role": "user", "content": f"{input_text}"},
    {"role": "assistant", "content": f"{summary_text}"},
    {"role": "user", "content": "중요한 키워드 5개를 뽑아주세요."},
    {"role": "assistant", "content": ""}
    ]

def extract_keywords_batch(batch):
    prompts = [ key_word_prompt(original, summary) for original, summary in zip(batch["original"], batch["summary"]) ]

    generated_texts = pipe(prompts, max_new_tokens=150, return_full_text=False)
    keywords = [ gen_text[0]["generated_text"] for gen_text in generated_texts ]
    batch["keywords"] = keywords
    return batch

Device set to use cuda:0


In [10]:
# dataset에 keyword 열 추가 (batch 단위로 처리)
sample_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
sample_dataset = sample_dataset.map(extract_keywords_batch, batched=True, batch_size=20)  # 배치사이즈 선택

sample_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Dataset({
    features: ['original', 'summary', 'keywords'],
    num_rows: 1000
})

In [11]:
# 데이터 전처리
def chat_keyword_summary_format(example):

        return [
            {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약 및 관련 키워드를 추출해주세요:\n{example['original']}"},
            {"role": "assistant", "content": f"한국어 요약:{example['summary']}\n키워드:{example['keywords']}"}
        ]

formatted = tokenizer.apply_chat_template(
    chat_keyword_summary_format(sample_dataset[0]), tokenize=False
)
print(formatted)

<bos><start_of_turn>user
다음 텍스트를 한국어로 간단히 요약 및 관련 키워드를 추출해주세요:
전두환 정권을 흔들었던 '큰 손' 장영자의 네 번째 유죄가 확정됐다.
  혐의는 사기, 징역은 4년이다.
  대법원 2부(주심 박상옥 대법관)는 9일 무죄를 주장한 장씨의 상고를 기각했다.
  장씨는 지인들에게서 세 차례에 걸쳐 6억여원을 가로채고 이들에게 위조 수표로 수억원을 요구한 혐의를 받는다.
  올해로 76세를 맞은 장씨는 이미 복역을 마친 29년을 포함해 인생의 절반가량인 33년을 감옥에서 보내게됐다.
   우수한 인재에게만 허락된다는 숙명여대 '5월의 여왕(메이퀸)' 출신인 장영자의 노년이 씁쓸하다.
  장씨는 전두환 전 대통령의 처삼촌인 고(故) 이규광씨의 처제이자, 중앙정보부 차장이었던 고(故) 이철희씨의 아내였다.
  전두환 정권 흔든 장영자의 삶장씨의 앞선 범행과 비교해 네 번째 사기 액수는 그 규모가 상당히 줄어든 편이다.
  장씨는 1980년대 전두환 전 대통령과 남편을 내세워 자금 압박에 시달리던 기업에 자금을 빌려준 뒤 몇 배에 달하는 어음을 할인 유통하며 이득을 챙겼다.
  그 규모가 총 7111억원에 달했다.
  장씨는 이중 6404억원의 어음을 할인해 사용했다.
   실체가 없던 어음은 부도가 났고 기업들은 도산했다.
  장씨는 이 첫 번째 사기로 15년형을 선고받았다.
  10년만인 1992년에 가석방됐다.
  전두환 전 대통령의 부인 이순자씨는 자서전 『당신은 외롭지 않다』에서 이를 "조금씩 민심도 안정되고 경제도 생기를 되찾아 (남편이) 자신감을 얻던 시점에 날벼락같이 찾아온 횡액과도 같은 사건"이라 회고했다.
  장씨가 당시 법정에서 아직 시중에 유통중인 어음이 있다며 "경제는 유통""나는 권력투쟁의 희생양이라"이라 했던 말은 유행어가 됐다.
  '장영자 어음할인 사건' 수사 축소 의혹으로 두 명의 법무부 장관이 경질됐다.
   "누나 성질 급하다" 70대 장영자의 돈 요구장씨는 출소한지 2년만인 1994년에 

In [12]:
EOS_TOKEN = tokenizer.eos_token  # 대화의 종료를 알리는 eos 토큰

def tokenize(element):
    formatted = tokenizer.apply_chat_template(
        chat_keyword_summary_format(element), tokenize=False  # 아직 eos 토큰이 포함되어 있지 않기 때문에 tokenize=False로 문자가 한 번에 숫자로 바뀌는 것을 방지
    ) + EOS_TOKEN  # eos 토큰 추가
    outputs = tokenizer(formatted)
    return {
        "input_ids": outputs["input_ids"],
        "attention_mask": outputs["attention_mask"],
    }

tokenized_sample_dataset = sample_dataset.map(tokenize)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
# 데이터셋 분리 및 콜레이터 설정
tokenized_sample_dataset = tokenized_sample_dataset.train_test_split(test_size=0.1, seed=42)
tokenized_sample_dataset

DatasetDict({
    train: Dataset({
        features: ['original', 'summary', 'keywords', 'input_ids', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['original', 'summary', 'keywords', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [14]:
response_template_ids = tokenizer.encode(
    "<start_of_turn>model\n",
    add_special_tokens=False
)

collator = DataCollatorForCompletionOnlyLM(
    response_template_ids, tokenizer=tokenizer
)

In [15]:
!wandb login

wandb: Currently logged in as: dgriii0606 (dgriii0606-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [57]:
# 학습 파라미터 설정
wandb.init(project="gemma-2B-it-Full-FIne-Tuning", entity="****")

training_args = TrainingArguments(
    output_dir="./keywords_gemma_results",  # 체크포인트 저장 위치
    max_steps=800,  # 몇 스텝을 학습할지 설정
    per_device_train_batch_size=2,  # 학습할 때, 한 번에 처리할 데이터 샘플 수
    per_device_eval_batch_size=4,   # 평가할 때, 한 번에 처리할 데이터 샘플 수
    warmup_steps=0,
    weight_decay=0.01,   # 모델 가중치를 조절하는 강도, 과적합 방지
    learning_rate=2e-4,  # 러닝레이트 
    logging_dir="./logs",  # 로그 파일 저장 위치
    logging_steps=100,  # 100스텝마다 학습이 어떻게 진행되고 있는지 기록함
    report_to="wandb",  # 학습 현황 관찰 도구
)

In [69]:
# 평가 메트릭 정의
bleu = evaluate.load("bleu")  # 주로 기계 번역 품질을 측정하는 지표
acc = evaluate.load("accuracy")  # 예측결과가 실제 정답고 얼마나 일치하는지 확인

def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        # 모델과 설정에 따라 logits에는 추가적인 텐서들이 포함될 수 있음
        # 예를 들어, past_key_values 같은 것들이 있을 수 있지만 logits는 항상 첫번째 요소임
        logits = logits[0]
    # 토큰 ID를 얻기 위해 argmax를 수행
    return logits.argmax(dim=-1)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # preds는 labels와 같은 형태를 가짐
    # preprocess_logits_for_metrics(logits, labels) 함수로 argmax(-1)가 계산된 후
    # 하지만 labels를 한 칸 이동해야 함
    labels = labels[:, 1:]
    preds = preds[:, :-1]

    # -100은 DataCollatorForCompletionOnlyLM에서 사용되는 ignore_index의 기본값
    mask = labels == -100  # 무시해야 할 토큰을 나타내는 특별한 값
    # -100을 토크나이저가 디코드할 수 있는 값으로 대체
    labels[mask] = tokenizer.pad_token_id
    preds[mask] = tokenizer.pad_token_id

    # bleu 점수는 텍스트를 입력으로 받기 때문에 토큰 ID에서 텍스트로 변환해야 됨
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    bleu_score = bleu.compute(predictions=decoded_preds, references=decoded_labels)

    # accuracy는 정수 리스트를 입력으로 받음  # -100이 아닌 부분만 평가하고 싶으므로 마스크에 부정(~)을 사용
    accuracy = acc.compute(predictions=preds[~mask], references=labels[~mask])

    return {**bleu_score, **accuracy}

In [38]:
## 참고서랑 다르게 Trainer에 'num_items_in_batch' 인자가 없는 이슈로 해당 인자를 받지않는 커스텀 트레이너 생성후 해당 커스텀 트레이너로 학습

In [70]:
from transformers import Trainer

# 1. 불필요한 인자(num_items_in_batch)를 자동으로 제거
# 2. 입력 데이터가 model.forward()가 받을 수 있는 인자만 유지하도록 변경
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """ 불필요한 인자 제거 및 텐서 변환 """
        inputs = {k: v for k, v in inputs.items() if k in model.forward.__code__.co_varnames}

        # 모델 실행
        outputs = model(**inputs)
        loss = outputs.loss

        return (loss, outputs) if return_outputs else loss

In [71]:
# 모델 학습 및 평가
trainer = CustomTrainer(
    args=training_args,
    model=model,
    tokenizer=tokenizer,
    data_collator=collator,
    train_dataset=tokenized_sample_dataset["train"],
    eval_dataset=tokenized_sample_dataset["test"],
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics,
    callbacks=[WandbCallback()]
)

/tmp/ipykernel_1690/3379539505.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
You are adding a <class 'transformers.integrations.integration_utils.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
WandbCallback


In [72]:
# GPU 메모리 캐시 정리
import torch
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()

/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:329: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [73]:
trainer.train()

Step,Training Loss
100,1.315600
200,1.444400
300,1.543400
400,1.508400
500,1.102800
600,0.697300
700,0.716000
800,0.832800


TrainOutput(global_step=800, training_loss=1.1450815296173096, metrics={'train_runtime': 254.4269, 'train_samples_per_second': 6.289, 'train_steps_per_second': 3.144, 'total_flos': 2.113522764418253e+16, 'train_loss': 1.1450815296173096, 'epoch': 1.7777777777777777})

In [75]:
trainer.evaluate()

{'eval_loss': 5.749093055725098,
 'eval_bleu': 0.08622845185337392,
 'eval_precisions': [0.29428172942817293,
  0.14332181337670258,
  0.06910147333471675,
  0.024793388429752067],
 'eval_brevity_penalty': 0.9352460710784825,
 'eval_length_ratio': 0.9372549019607843,
 'eval_translation_length': 5019,
 'eval_reference_length': 5355,
 'eval_accuracy': 0.3418083130981288,
 'eval_runtime': 3.5201,
 'eval_samples_per_second': 28.408,
 'eval_steps_per_second': 7.102,
 'epoch': 1.7777777777777777}

In [77]:
input_text = """
다음 텍스트를 한국어로 간단히 요약해주세요: \n 김해국제공항에서 발생한 에어부산(BX391편) 여객기 화재 사고 원인을 규명하기 위한 조사가 한창인 가운데 에어부산이
기내 화재 위험 최소화 정책을 발표했다. 해당 정책에는 휴대 수하물 내 보조배터리 소지 유무를 사전 확인·통제하고 안내방송 확대와 객실승무원 훈련을 강화하는 내용 등이 담겼다.
에어부산은 4일 보조배터리 기내 탑재 사례가 증가하며 발열 등 이상 현상 발생 빈도가 높아진 데 따라 이 같은 정책을 수립했다고 밝혔다. 에어부산이 내놓은 정책에서 주목할 부분은
휴대 수하물 내 보조배터리 소지 유무를 사전 확인하는 절차다. 에어부산은 탑승구에서 휴대 수하물 내 배터리 소지 유무를 사전 확인하고, 기내에서는 탑승구에서 확인을 완료한 수하물만
선반에 보관될 수 있도록 통제를 강화할 방침이다. 이는 탑승구에서 휴대 수하물 내 배터리가 있는지를 점검해 스티커나 택(TAG) 등의 별도 표식을 부착하고 기내에서는 표식이 부착된
수하물만 선반 보관이 가능하도록 허용하겠다는 의미다. 해당 정책은 오는 7일부터 일부 노선에서 시범 운영을 거쳐 전 노선으로 확대 운영할 계획이다. 이 외에도 예약·발권과 탑승수속
단계에서 기존 안내에 더해 보조배터리 기내 선반 탑재 금지에 대한 동의 절차를 시행하고, 출발 1일 전 예약 고객 대상 별도의 안내 문자를 발송하는 등 안내도 강화한다.
기내 안내 방송도 기존 2회에서 3회로 확대 시행하기로 했다. 에어부산은 한층 구체적이고 명확한 내용으로 기내 안내 방송문을 보완해 지난달 말부터 변경 적용한 바 있다.
이 외에도 객실승무원 훈련도 강화할 방침이다. 또 배터리 화재 발생 시 열폭주와 폭발 위험을 최소화할 수 있는 장비(Lithium battery fire containment pouch)와 방화장갑
등을 구매·구비할 계획도 밝혔다. 최근 조사당국은 화재 원인을 규명하기 위해 합동 감식에 나서는 등 사고 관련 조사 진행이 한창이다. 항공·철도사고조사위원회와 프랑스
사고조사당국(BEA), 국립과학수사연구원, 경찰 과학수사대, 소방으로 구성된 합동조사팀은 현장에서 수집한 증거물을 국립과학수사연구원 등으로 옮겨 정밀 감식을 진행 중인 것으로
알려졌다. 이 외에도 사조위는 관계자와 목격자 진술을 받고 폐쇄회로(CC)TV 영상과 블랙박스를 분석하는 등의 조사를 진행 중이다. 정병섭 에어부산 대표이사는
"기내 화재에 대한 손님들의 우려와 불안을 해소하고 안전한 여행 환경을 확보하기 위해 에어부산이 선제적으로 강화 정책을 수립하게 됐다"며 "사내 종사자들의 안전의식 고취와
역량 강화를 비롯해 항공 안전 문화 확산을 위해서도 지속적으로 노력해 나가겠다"고 밝혔다. 앞서 지난달 28일 오후 부산에서 홍콩으로 운항하려던 에어부산 BX391편은 김해공항
계류장에 머무르던 중 항공기 내부 뒤편에서 불이 나기 시작해 동체 상부를 태웠다. 당시 탑승했던 170명 승객과 6명의 승무원은 비상 슬라이드를 통해 탈출했으며,
이 과정에서 7명이 경상을 입었다.
"""

def get_chat_format(input_text):

        return [
            {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약 및 관련 키워드를 추출해주세요:\n{input_text}"},
            {"role": "assistant", "content": "한국어 요약:\n키워드:"}
        ]


def change_inference_chat_format(input_text):
    return [
    {"role": "user", "content": f"{input_text}"},
    {"role": "assistant", "content": ""}
    ]
prompt = change_inference_chat_format(input_text)

# tokenizer 초기화 및 적용
inputs = tokenizer.apply_chat_template(prompt, tokenize=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
outputs = model.generate(inputs, max_new_tokens=512, use_cache=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))  ## 오히려 미세 조정하기 전이 더 출력 결과가 올바름..ㅜㅜ

user
다음 텍스트를 한국어로 간단히 요약해주세요: 
 김해국제공항에서 발생한 에어부산(BX391편) 여객기 화재 사고 원인을 규명하기 위한 조사가 한창인 가운데 에어부산이
기내 화재 위험 최소화 정책을 발표했다. 해당 정책에는 휴대 수하물 내 보조배터리 소지 유무를 사전 확인·통제하고 안내방송 확대와 객실승무원 훈련을 강화하는 내용 등이 담겼다.
에어부산은 4일 보조배터리 기내 탑재 사례가 증가하며 발열 등 이상 현상 발생 빈도가 높아진 데 따라 이 같은 정책을 수립했다고 밝혔다. 에어부산이 내놓은 정책에서 주목할 부분은
휴대 수하물 내 보조배터리 소지 유무를 사전 확인하는 절차다. 에어부산은 탑승구에서 휴대 수하물 내 배터리 소지 유무를 사전 확인하고, 기내에서는 탑승구에서 확인을 완료한 수하물만
선반에 보관될 수 있도록 통제를 강화할 방침이다. 이는 탑승구에서 휴대 수하물 내 배터리가 있는지를 점검해 스티커나 택(TAG) 등의 별도 표식을 부착하고 기내에서는 표식이 부착된
수하물만 선반 보관이 가능하도록 허용하겠다는 의미다. 해당 정책은 오는 7일부터 일부 노선에서 시범 운영을 거쳐 전 노선으로 확대 운영할 계획이다. 이 외에도 예약·발권과 탑승수속
단계에서 기존 안내에 더해 보조배터리 기내 선반 탑재 금지에 대한 동의 절차를 시행하고, 출발 1일 전 예약 고객 대상 별도의 안내 문자를 발송하는 등 안내도 강화한다.
기내 안내 방송도 기존 2회에서 3회로 확대 시행하기로 했다. 에어부산은 한층 구체적이고 명확한 내용으로 기내 안내 방송문을 보완해 지난달 말부터 변경 적용한 바 있다.
이 외에도 객실승무원 훈련도 강화할 방침이다. 또 배터리 화재 발생 시 열폭주와 폭발 위험을 최소화할 수 있는 장비(Lithium battery fire containment pouch)와 방화장갑
등을 구매·구비할 계획도 밝혔다. 최근 조사당국은 화재 원인을 규명하기 위해 합동 감식에 나서는 등 사고 관련 조사 진행이 한창이다. 항공·철도사고조사위원회와 프랑스
사고조사당국